In [165]:
%cd ../..

c:\


# Creating Baseline forecasts with NIXTLA

In [166]:
import numpy as np
import pandas as pd
import time
import os
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly_resampler
from itertools import cycle

pio.templates.default = "plotly_white"
import warnings
import humanize

from functools import partial
from statsforecast.core import StatsForecast
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    WindowAverage,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    HoltWinters,
    ETS,
    AutoETS,
    AutoARIMA,
    ARIMA,
    AutoTheta,
    DynamicTheta,
    DynamicOptimizedTheta,
    Theta,
    OptimizedTheta,
    TBATS,
    AutoTBATS,
    MSTL

)
from datasetsforecast.losses import *
from src.utils.ts_utils import forecast_bias

import time
from src.utils import plotting_utils

from tqdm import tqdm 
np.random.seed(42)
tqdm.pandas()

In [167]:
# this makes it so that the outputs of the predict methods have the id as a column 
# instead of as the index
if 'NIXTLA_ID_AS_COL' in os.environ:
    del os.environ['NIXTLA_ID_AS_COL']
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [168]:
os.makedirs("imgs/chapter_4", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")

In [169]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", title="", font_size=15):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title_text=title,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            legend=dict(
                font=dict(size=font_size),
                orientation="h",
                yanchor="bottom",
                y=0.98,
                xanchor="right",
                x=1,
            ),
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=font_size),
                tickfont=dict(size=font_size),
            )
        )
    return fig

In [170]:
import plotly.express as px

def plot_forecast(pred_df, forecast_columns, timestamp_col, forecast_display_names=None):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)
    
    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = [c.replace("rgb", "rgba").replace(")", ", <alpha>)") for c in px.colors.qualitative.Dark2]
    act_color = colors[0]
    colors = cycle(colors[1:])
    dash_types = cycle(["dash", "dot", "dashdot"])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df[mask]['energy_consumption'],
                             mode='lines', line=dict(color=act_color.replace("<alpha>", "0.3")),
                             name='Actual Consumption'))
    
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(go.Scatter(x=pred_df[mask][timestamp_col], y=pred_df.loc[mask, col],
                                 mode='lines', line=dict(dash=next(dash_types), color=next(colors).replace("<alpha>", "1")),
                                 name=display_col))
    return fig

In [171]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed.parquet")
    train_df = train_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed.parquet")
    val_df = val_df[['LCLid',"timestamp","energy_consumption","frequency"]]
    test_df = pd.read_parquet(preprocessed/"selected_blocks_test_missing_imputed.parquet")
    test_df = test_df[['LCLid',"timestamp","energy_consumption","frequency"]]
except FileNotFoundError:
    print(f"Warning: File not found in {preprocessed}. Ensure you've run '01-Setting up Experiment Harness.ipynb' in Chapter 04 and that the file path is correct.")

In [172]:
print("Min train_df Date: " , train_df.timestamp.min())
print("Max train_df Date: " , train_df.timestamp.max())
print("Min val_df Date: " , val_df.timestamp.min())
print("Max val_df Date: " , val_df.timestamp.max())
print("Min test_df Date: " , test_df.timestamp.min())
print("Max test_df Date: " , test_df.timestamp.max())

Min train_df Date:  2012-01-01 00:30:00
Max train_df Date:  2013-12-31 23:30:00
Min val_df Date:  2014-01-01 00:00:00
Max val_df Date:  2014-01-31 23:30:00
Min test_df Date:  2014-02-01 00:00:00
Max test_df Date:  2014-02-27 23:30:00


In [173]:
train_df = train_df[train_df.timestamp >'2012-01-01']
print("Min train_df Date: " , train_df.timestamp.min())

Min train_df Date:  2012-01-01 00:30:00


In [174]:
#picking a single time series from the dataset for illustration
freq = train_df.iloc[0]['frequency']
ts_train = train_df.loc[train_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_val = val_df.loc[val_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]
ts_test = test_df.loc[test_df.LCLid=="MAC000193", ['LCLid',"timestamp","energy_consumption"]]


# Baseline Forecasts

In [175]:
pred_df = pd.concat([ts_train, ts_val])


In [176]:
def evaluate_performance(ts_train, ts_test, models, metrics, freq, level, id_col, time_col, target_col, h, metric_df=None):
    if metric_df is None:
        metric_df = pd.DataFrame()  # Initialize an empty DataFrame if not provided

    results = ts_test.copy()

    # Timing dictionary to store train and predict durations
    timing = {}

    for model in models:
        model_name = model.__class__.__name__
        evaluation = {}  # Reset the evaluation dictionary for each model

        # Start the timer for fitting and prediction
        start_time = time.time()

        # Instantiate StatsForecast class
        sf = StatsForecast(
            models=[model],
            freq=freq,
            n_jobs=-1,
            fallback_model=Naive()
        )

        # Efficiently predict without storing memory
        y_pred = sf.forecast(
            h=h,
            df=ts_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        )

        # Calculating the duration
        duration = time.time() - start_time
        timing[model_name] = duration

        # Merge prediction results to the original dataframe
        results = results.merge(y_pred, how='left', on=[id_col, time_col])

        ids = ts_train[id_col].unique()
        # Calculate metrics
        for id in ids:
            temp_results = results[results[id_col] == id]
            temp_train = ts_train[ts_train[id_col] == id]
            for metric in metrics:
                metric_name = metric.__name__
                if metric_name == 'mase':
                    evaluation[metric_name] = metric(temp_results[target_col].values,
                                                    temp_results[model_name].values,
                                                    temp_train[target_col].values, seasonality=48)
                else:
                    evaluation[metric_name] = metric(temp_results[target_col].values, temp_results[model_name].values)
            evaluation[id_col] = id
            evaluation['Time Elapsed'] = timing[model_name]

            # Prepare and append this model's results to metric_df
            temp_df = pd.DataFrame(evaluation, index=[0])
            temp_df['Model'] = model_name
            metric_df = pd.concat([metric_df, temp_df], ignore_index=True)

    return results, metric_df

# NAIVE FORECAST

In [177]:
metrics = pd.DataFrame()

results, metrics = evaluate_performance(
    ts_train=ts_train, 
    ts_test=ts_val, 
    models=[Naive()], 
    metrics=[mase, mae, mse, rmse, smape, forecast_bias], 
    freq=freq,
    level=[],  # Ensure this is correct or adjust as necessary
    id_col='LCLid',
    time_col='timestamp',
    target_col='energy_consumption',
    h=len(ts_val),
    metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
)

In [178]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.58813,74.340254,MAC000193,0.154695,Naive


In [179]:
model_name = ['Naive']
model_display_name = ['Naive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/naive.png")
fig.show()

# SEASONAL NAIVE FORECAST

In [180]:
results, metrics = (
    evaluate_performance(
        ts_train=ts_train, 
        ts_test=ts_val, 
        models = [SeasonalNaive(season_length=48*7)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)


In [181]:
results.head()

,LCLid,timestamp,energy_consumption,SeasonalNaive
0,MAC000193,2014-01-01 00:00:00,0.223,0.026
1,MAC000193,2014-01-01 00:30:00,0.274,0.041
2,MAC000193,2014-01-01 01:00:00,0.308,0.019
3,MAC000193,2014-01-01 01:30:00,0.279,0.011
4,MAC000193,2014-01-01 02:00:00,0.000,0.041


In [182]:
model_name = ['SeasonalNaive']
model_display_name = ['SeasonalNaive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/seasonal_naive.png")
fig.show()

## Moving Average Forecast

In [183]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [WindowAverage(window_size = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)


In [184]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.154695,Naive
1,1.500943,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.124430,SeasonalNaive
2,1.857255,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.105527,WindowAverage


In [185]:
model_name = ['WindowAverage']
model_display_name = ['WindowAverage']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/window_average.png")
fig.show()

# Exponential Smoothing Forecast

There are several forecast of exponential smoothing as discussed in the chapter.   NIXTLA allows you do specify the type of exponential smoothing you would like to do.  Or, you can use the ETS function which will test which version of exponential smoothing best fits your data and automatically finds the best parameters.

In [186]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ HoltWinters(error_type = 'A', season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)


In [187]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.154695,Naive
1,1.500943,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.124430,SeasonalNaive
2,1.857255,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.105527,WindowAverage
3,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,38.611616,HoltWinters


In [188]:
model_name = ['HoltWinters']
model_display_name = ['HoltWinters']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

In [189]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ AutoETS(model = 'AAA',season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)


In [190]:
results.head()

,LCLid,timestamp,energy_consumption,AutoETS
0,MAC000193,2014-01-01 00:00:00,0.223,0.207815
1,MAC000193,2014-01-01 00:30:00,0.274,0.033957
2,MAC000193,2014-01-01 01:00:00,0.308,0.014500
3,MAC000193,2014-01-01 01:30:00,0.279,-0.019726
4,MAC000193,2014-01-01 02:00:00,0.000,-0.016634


In [191]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.154695,Naive
1,1.500943,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.124430,SeasonalNaive
2,1.857255,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.105527,WindowAverage
3,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,38.611616,HoltWinters
4,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,24.584767,AutoETS


In [192]:
model_name = ['AutoETS']
model_display_name = ['AutoETS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ets.png")
fig.show()

## ARIMA

Not using AutoARIMA because it just takes too much time for long time series
Sample AutoARIMA is given in this notebook.

In [193]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ ARIMA(order = (2,1,1), seasonal_order = (1,1,1), season_length = 48)],
        #models = [ ARIMA(order = (0,1,2), seasonal_order = (0,0,2), season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [194]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.154695,Naive
1,1.500943,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.124430,SeasonalNaive
2,1.857255,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.105527,WindowAverage
3,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,38.611616,HoltWinters
4,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,24.584767,AutoETS
5,1.212629,0.203529,0.106482,0.326316,98.716784,25.337937,MAC000193,22.628870,ARIMA


In [195]:
model_name = ['ARIMA']
model_display_name = ['ARIMA']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/ARIMA.png")
fig.show()

## Sample Auto ARIMA
Note:  This may take a long time to run.

To get a better understanding of what parameters to use, you can run AutoARIMA and output the fitted parameters.  AutoARIMA however is very slow, so I have includedd a sample output from AutoARIMA.

In [196]:
# sf = StatsForecast(
#     models=[AutoARIMA( max_p = 2, max_d=1, max_q = 2, max_P=2, max_D = 1, max_Q = 2, 
#                       start_p = 1, start_q = 1, start_P = 1, start_Q = 1, stepwise = True, season_length=48)],
#     freq=freq,
#     n_jobs=-1,
#     fallback_model = Naive()
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

#### Sample AutoARIMA Output

note the ARMA section
arma': (0, 2, 0, 2, 48, 1, 0)

This correspondes to:  p, q, P, Q, s, d, D 

p = 0, d = 1, q = 2

P = 0, D = 0, Q = 2

These parameters however could still be further tuned but due to the timing, will leave that as an exercise.  Better parameters include above in our ARIMA model.

In [197]:
# {'coef': {'ma1': -0.3800676288610244,
#   'ma2': -0.25121322485842584,
#   'sma1': 0.1300139905884248,
#   'sma2': 0.1045851891918472},
#  'sigma2': 0.05918752833035802,
#  'var_coef': array([[ 7.04752468e-07,  9.11329831e-07,  2.55693473e-07,
#          -1.87114423e-06],
#         [ 9.11329831e-07,  7.41876851e-06, -3.18464778e-07,
#          -8.00782484e-06],
#         [ 2.55693473e-07, -3.18464778e-07,  5.50841803e-07,
#          -4.88097177e-07],
#         [-1.87114423e-06, -8.00782484e-06, -4.88097177e-07,
#           1.03626760e-05]]),
#  'mask': array([ True,  True,  True,  True]),
#  'loglik': -188.2911979664168,
#  'aic': 386.5823959328336,
#  'arma': (0, 2, 0, 2, 48, 1, 0),
#  'residuals': array([ 0.000368  ,  0.01585512, -0.19137661, ..., -0.3180852 ,
#         -0.35435519, -0.1594367 ]),
#  'code': 2,
#  'n_cond': 0,
#  'nobs': 35087,
# ...
#  'bic': 428.9103257852313,
#  'aicc': 386.58410626036135,
#  'ic': None,
#  'xreg': None,
#  'x': array([0.368, 0.386, 0.17 , ..., 0.147, 0.111, 0.09 ], dtype=float32),
#  'lambda': None}

## Theta
THETA method in NIXTLA can be a little slow due to it running tests on which is the optimal theta parameter. 

In [198]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ Theta(season_length =48, decomposition_type = 'additive' )], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [199]:
model_name = ['Theta']
model_display_name = ['Theta']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_theta.png")
fig.show()

In [200]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.154695,Naive
1,1.500943,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.124430,SeasonalNaive
2,1.857255,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.105527,WindowAverage
3,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,38.611616,HoltWinters
4,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,24.584767,AutoETS
5,1.212629,0.203529,0.106482,0.326316,98.716784,25.337937,MAC000193,22.628870,ARIMA
6,1.428325,0.239731,0.167725,0.409542,104.389441,56.917018,MAC000193,105.191291,Theta


## TBATS
NIXTLA again offers TBATS and AutoTBATS.  AutoTBATS can be slow, so here we will just do TBATS. Similar to AutoARIMA, you can run a model using AutoTBATS to get a recommendation of good parameters to use.


In [201]:
# sf = StatsForecast(
#     models=[TBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
#     freq=freq,
#     n_jobs=2
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [202]:
# sf = StatsForecast(
#     models=[AutoTBATS(seasonal_periods  = 48, use_trend=True, use_damped_trend=True)],
#     freq=freq,
#     n_jobs=2
# )

# y_pred = sf.fit( 
                  
#                     df=ts_train,        
#                     id_col = 'LCLid',
#                     time_col = 'timestamp',
#                     target_col = 'energy_consumption',

#                     )
# sf.fitted_[0,0].model_

In [203]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ TBATS(seasonal_periods  = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

c:\Users\tacke\anaconda3\envs\modern_ts_2E\Lib\site-packages\statsforecast\tbats.py:944: UserWarning:

The time series is trend-stationary, disabling trend components.



In [204]:
metrics

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819549,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.154695,Naive
1,1.500943,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.124430,SeasonalNaive
2,1.857255,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.105527,WindowAverage
3,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,38.611616,HoltWinters
4,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,24.584767,AutoETS
5,1.212629,0.203529,0.106482,0.326316,98.716784,25.337937,MAC000193,22.628870,ARIMA
6,1.428325,0.239731,0.167725,0.409542,104.389441,56.917018,MAC000193,105.191291,Theta
7,1.290473,0.216594,0.111659,0.334154,94.634706,18.834032,MAC000193,8.483486,TBATS


In [205]:
model_name = ['TBATS']
model_display_name = ['TBATS']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

## MSTL

In [206]:
results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [ MSTL(season_length = 48)], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

In [207]:
model_name = ['MSTL']
model_display_name = ['MSTL']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='timestamp')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MSE: {metrics.loc[metrics.Model==model_name[0]][['mse']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
#fig.write_image("imgs/chapter_4/auto_tbats.png")
fig.show()

In [208]:
metric_styled = metrics.reset_index(drop=True).style.format({
            "mae": "{:.3f}", 
            "mse": "{:.3f}", 
            "mase": "{:.3f}", 
            "rmse": "{:.3f}", 
            "smape": "{:.3f}",            
            "forecast_bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["mae","mse","mase","rmse","smape","Time Elapsed"])
display(metric_styled)

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.820,0.305,0.249,0.499,113.588,74.34%,MAC000193,0.154695,Naive
1,1.501,0.252,0.191,0.437,78.739,13.74%,MAC000193,0.124430,SeasonalNaive
2,1.857,0.312,0.183,0.428,100.698,11.51%,MAC000193,0.105527,WindowAverage
3,1.139,0.191,0.101,0.318,88.890,10.44%,MAC000193,38.611616,HoltWinters
4,1.139,0.191,0.101,0.318,88.890,10.44%,MAC000193,24.584767,AutoETS
5,1.213,0.204,0.106,0.326,98.717,25.34%,MAC000193,22.628870,ARIMA
6,1.428,0.240,0.168,0.410,104.389,56.92%,MAC000193,105.191291,Theta
7,1.290,0.217,0.112,0.334,94.635,18.83%,MAC000193,8.483486,TBATS
8,1.506,0.253,0.148,0.385,117.314,48.11%,MAC000193,4.904804,MSTL


Based on this sample test, the best performing models are (HoltWinters, and ETS), ARIMA, and TBATS.  Lets build that for all models using AutoETS and TBATS.  ARIMA gives similar performance to TBATS, but TBATS is faster. 

# VALIDATION SET
## Running Baseline Forecast for all consumers

In [209]:
ids = list(train_df.LCLid.unique()[:500]) # slicing dataframe for the sake of time.  May want to consider setting this low if working on a slower machine

train_df = train_df[train_df.LCLid.isin(ids)]
val_df = val_df[val_df.LCLid.isin(ids)]
test_df = test_df[test_df.LCLid.isin(ids)]

print("Length of validation Data: ", len(val_df[val_df.LCLid =='MAC000948'].LCLid))

Length of validation Data:  1488


### Forecasting for Validation Period for TBATS and AutoETS

In [210]:
validation_models =  [AutoETS(model = 'AAA',season_length = 48), 
                      TBATS(seasonal_periods  = 48)]
#validation_models = [Naive()]
validation_models_names = [model.__class__.__name__ for model in validation_models]
metric_df = pd.DataFrame([])
h_val = 1488

In [211]:
aggval_metrics = pd.DataFrame()

baseline_val_pred_df, aggval_metrics = (
    evaluate_performance(
        train_df[["LCLid","timestamp","energy_consumption"]], 
        val_df[["LCLid","timestamp","energy_consumption"]], 
        models =validation_models, 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = h_val,
        metric_df = aggval_metrics
        )
)

#### Evaluation of Baseline Forecast

In [255]:
aggval_metrics.head()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.041762,0.167747,0.050541,0.224814,59.395635,-26.861671,MAC000768,1784.020385,AutoETS
1,0.765181,0.107071,0.034325,0.185269,34.775889,17.419682,MAC000948,1784.020385,AutoETS
2,1.055397,0.184768,0.051710,0.227398,42.697275,-31.349871,MAC003299,1784.020385,AutoETS
3,1.011995,0.142319,0.040731,0.201820,30.385882,-5.956137,MAC003157,1784.020385,AutoETS
4,1.138965,0.191165,0.100847,0.317565,88.889992,10.440250,MAC000193,1784.020385,AutoETS


In [239]:
autoets_val_metric_df = aggval_metrics[aggval_metrics.Model =='AutoETS']
overall_metrics_val_autoets = {
    "Algorithm": "AutoETS",
    "MAE": mae(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "MSE": mse(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "meanMASE": autoets_val_metric_df.mase.mean(),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values)
}
overall_metrics_val_autoets

{'Algorithm': 'AutoETS',
 'MAE': 0.119717814,
 'MSE': 0.057770558,
 'meanMASE': 1.0284381,
 'Forecast Bias': 4.2695797979831696}

In [240]:
tbats_val_metric_df = aggval_metrics[aggval_metrics.Model =='TBATS']
overall_metrics_val_tbats = {
    "Algorithm": "TBATS",
    "MAE": mae(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.TBATS.values),
    "MSE": mse(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.TBATS.values),
    "meanMASE": tbats_val_metric_df.mase.mean(),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.TBATS.values)
}
overall_metrics_val_tbats

{'Algorithm': 'TBATS',
 'MAE': 0.1561929,
 'MSE': 0.08932494,
 'meanMASE': 1.3848531,
 'Forecast Bias': 6.980016082525253}

In [241]:
baseline_val_metrics_df = pd.DataFrame([overall_metrics_val_autoets, overall_metrics_val_tbats])

display(baseline_val_metrics_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]))

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,AutoETS,0.120,0.058,1.028,4.27%
1,TBATS,0.156,0.089,1.385,6.98%


Here we are importing metrics meant to be able to efficiently cycle through each local forecast. We will rename them to not have any conflicts with our other error metrics.

In both the below validation and testing datasets, we will calculate both the local and global accuracy for each model type.

In [285]:
baseline_val_metrics_df.head()

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,AutoETS,0.119718,0.057771,1.028438,4.269580
1,TBATS,0.156193,0.089325,1.384853,6.980016


In [262]:
fig = px.histogram(aggval_metrics, 
                   x="mase", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mase", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,7])
fig.write_image("imgs/chapter_4/mase_dist.png")
fig.show()

In [272]:
fig = px.histogram(aggval_metrics, 
                   x="forecast_bias", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=250, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="forecast_bias", ylabel="Probability Density", title="Distribution of BIAS in the dataset")
fig.update_layout(xaxis_range=[-200,200])
fig.write_image("imgs/chapter_4/bias_dist.png")
fig.show()

In [251]:
fig = px.histogram(aggval_metrics, 
                   x="mae", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mae", ylabel="Probability Density", title="Distribution of MAE in the dataset")
fig.update_layout(xaxis_range=[0,1])
fig.write_image("imgs/chapter_4/mae_dist.png")
fig.show()

In [271]:
fig = px.histogram(aggval_metrics, 
                   x="mse", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="mse", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,1.5])
fig.write_image("imgs/chapter_4/mse_dist.png")
fig.show()

#### Saving the Baseline Forecasts and Metrics

In [81]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [82]:
baseline_val_pred_df.to_pickle(output/"baseline_val_prediction_df.pkl")
baseline_val_metrics_df.to_pickle(output/"baseline_val_metrics_df.pkl")
aggval_metrics.to_pickle(output/"baseline_val_aggregate_metrics.pkl")

# TEST SET
### Forecasting for Test Period

In [ ]:
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse as rmse_local
from utilsforecast.losses import mae as mae_local
from utilsforecast.losses import mse as mse_local
from utilsforecast.losses import mase as mase_local

from functools import partial

In [83]:
_train_df = pd.concat([train_df, val_df])
print("Length of training Data: ", len(test_df[test_df.LCLid =='MAC000948'].LCLid))
test_df.tail()

Length of training Data:  1296


,LCLid,timestamp,energy_consumption,frequency
33259,MAC004796,2014-02-27 21:30:00,0.244,30min
33260,MAC004796,2014-02-27 22:00:00,0.216,30min
33261,MAC004796,2014-02-27 22:30:00,0.211,30min
33262,MAC004796,2014-02-27 23:00:00,0.116,30min
33263,MAC004796,2014-02-27 23:30:00,0.441,30min


In [84]:
test_models =  [ AutoETS(model = 'AAA',season_length = 48), TBATS(seasonal_periods  = 48)]
h_test = 1296

In [85]:
sf = StatsForecast(
    models=test_models,
    freq=freq,
    n_jobs=-1,
    fallback_model= SeasonalNaive(season_length=48)
)

# sf.fit( df = _train_df[['timestamp', 'LCLid', 'energy_consumption']] ,    
#         id_col = 'LCLid',
#         time_col = 'timestamp',
#         target_col = 'energy_consumption',
#         )

# baseline_test_pred_df = sf.predict( h =1296 )

# Memory efficient predictions
baseline_test_pred_df = sf.forecast(df=_train_df, 
                                        h=h_test, 
                                        level=[],   
                                        id_col = 'LCLid',
                                        time_col = 'timestamp',
                                        target_col = 'energy_consumption',
        )
baseline_test_pred_df = pd.merge(baseline_test_pred_df, test_df[['timestamp', 'LCLid', 'energy_consumption']], on = ['LCLid','timestamp'], how = 'left')

In [86]:
baseline_test_pred_df.head()

,LCLid,timestamp,AutoETS,TBATS,energy_consumption
0,MAC000061,2014-02-01 00:00:00,0.056101,0.015,0.066
1,MAC000061,2014-02-01 00:30:00,0.039241,0.064,0.063
2,MAC000061,2014-02-01 01:00:00,0.024739,0.060,0.040
3,MAC000061,2014-02-01 01:30:00,0.022753,0.059,0.020
4,MAC000061,2014-02-01 02:00:00,0.023229,0.042,0.018


In [87]:
fcst_mase = partial(mase_local, seasonality=48)

baseline_test_metrics_df = evaluate(baseline_test_pred_df, 
        metrics=[rmse_local, mae_local, mse_local,fcst_mase],  
        train_df = _train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption'
        )



In [88]:
baseline_test_metrics_df.head()

,LCLid,metric,AutoETS,TBATS
0,MAC000061,rmse,0.078926,0.103424
1,MAC000062,rmse,0.169500,0.166167
2,MAC000066,rmse,0.068317,0.062972
3,MAC000086,rmse,0.168439,0.186434
4,MAC000126,rmse,0.104435,0.105968


In [89]:
cols_to_exclude  = ['LCLid', 'metric']
test_columns = [col for col in baseline_test_metrics_df.columns if col not in cols_to_exclude]
test_columns

['AutoETS', 'TBATS']

In [90]:
baseline_test_metrics_df = pd.melt(baseline_test_metrics_df, id_vars = ['LCLid','metric'],value_vars= test_columns, var_name='Algorithm' , value_name='value')
baseline_test_metrics_df = baseline_test_metrics_df.pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'value').reset_index()
baseline_test_metrics_df.head()

C:\Users\tacke\AppData\Local\Temp\ipykernel_26196\1357591231.py:2: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



metric,LCLid,Algorithm,mae,mase,mse,rmse
0,MAC000061,AutoETS,0.058033,0.994184,0.006229,0.078926
1,MAC000061,TBATS,0.075398,1.291667,0.010697,0.103424
2,MAC000062,AutoETS,0.091553,1.020420,0.028730,0.169500
3,MAC000062,TBATS,0.075708,0.843818,0.027612,0.166167
4,MAC000066,AutoETS,0.044511,0.844105,0.004667,0.068317


In [91]:
validation_models_names

['AutoETS', 'TBATS']

In [274]:
baseline_test_metrics_df.head()

metric,LCLid,Algorithm,mae,mase,mse,rmse
0,MAC000061,AutoETS,0.058033,0.994184,0.006229,0.078926
1,MAC000061,TBATS,0.075398,1.291667,0.010697,0.103424
2,MAC000062,AutoETS,0.091553,1.020420,0.028730,0.169500
3,MAC000062,TBATS,0.075708,0.843818,0.027612,0.166167
4,MAC000066,AutoETS,0.044511,0.844105,0.004667,0.068317


In [276]:
from datasetsforecast.losses import *

agg_test_metrics = []  # Initialize an empty list to store the metrics dictionaries

for model in validation_models_names:
    actual_series = baseline_test_pred_df['energy_consumption'].values
    pred_series = baseline_test_pred_df[model].values
    
    # Create a dictionary for the current model's metrics
    agg_test_metrics1 = {
        "Algorithm": model,
        "MAE": mae(actual_series, pred_series),
        "MSE": mse(actual_series, pred_series),
        "meanMASE": baseline_test_metrics_df[baseline_test_metrics_df.Algorithm == model].mase.mean(),
        "Forecast Bias": forecast_bias(actual_series, pred_series),
    }
    
    # Append the dictionary to the list
    agg_test_metrics.append(agg_test_metrics1)
agg_test_metrics_df = pd.DataFrame(agg_test_metrics)
agg_test_metrics_df

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,AutoETS,0.119328,0.060436,0.997025,-9.875312
1,TBATS,0.143499,0.095902,1.175378,-11.635356


### Evaluation of Baseline Forecast

In [277]:
display(agg_test_metrics_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]))

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,AutoETS,0.119,0.060,0.997,-9.88%
1,TBATS,0.143,0.096,1.175,-11.64%


In [280]:
fig = px.histogram(baseline_test_metrics_df, 
                   x="mase", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,10])
fig.write_image("imgs/chapter_4/mase_dist_test.png")
fig.show()

In [281]:
fig = px.histogram(baseline_test_metrics_df, 
                   x="mae", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
fig.write_image("imgs/chapter_4/mae_dist_test.png")
fig.update_layout(xaxis_range=[0,1.1])
fig.show()

In [282]:
fig = px.histogram(baseline_test_metrics_df, 
                   x="mse", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,1])
fig.write_image("imgs/chapter_4/mse_dist_test.png")
fig.show()

### Saving the Baseline Forecasts and Metrics

In [283]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [284]:
baseline_test_pred_df.to_pickle(output/"baseline_test_prediction_df.pkl")
baseline_test_metrics_df.to_pickle(output/"baseline_test_metrics_df.pkl")
agg_test_metrics_df.to_pickle(output/"baseline_test_aggregate_metrics.pkl")